# Building Agents

시작하기 전에 [이 슬라이드](https://docs.google.com/presentation/d/13c0L1CQWAL7fuCXakOqjkvoodfynPJI4Hw_4H76okVU/edit?usp=sharing)를 참고하여 배경 지식을 확인하세요!

이메일 어시스턴트를 처음부터 구축해 보겠습니다. 
1) 랭그래프로 에이전트 아키텍처 설계
2) 랭스미스로 테스트
3) 휴먼 인 더 루프(Human-in-the-loop), 
4) 메모리 순으로 진행합니다. 

이 다이어그램은 각 구성 요소가 어떻게 결합되는지 보여줍니다:

![overview-img](assets/overview.png)

## Resources

- Notebook Reference: [agent.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/agent.ipynb)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)



## 환경 설정

In [1]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)

True

## 도구 정의

이메일 어시스턴트가 사용할 간단한 도구들을 `@tool` 데코레이터로 정의합니다.

In [2]:
from datetime import datetime

from langchain_core.tools import tool
from pydantic import BaseModel


@tool
def write_email(to: str, subject: str, content: str) -> str:
    """이메일을 작성하여 보냅니다."""
    # 임시 응답 - 실제 앱에서는 이메일을 발송합니다
    return f"{to}에게 발송된 이메일, 제목: '{subject}', 내용: {content}"


@tool
def schedule_meeting(
    attendees: list[str],
    subject: str,
    duration_minutes: int,
    preferred_day: datetime,
    start_time: int,
) -> str:
    """캘린더에 회의를 예약합니다."""
    # 임시 응답 - 실제 앱에서는 캘린더와 일정을 확인합니다
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"'{subject}' 회의가 {date_str} {start_time}에 {duration_minutes}분 동안 {len(attendees)}명의 참석자와 함께 예정되어 있습니다"


@tool
def check_calendar_availability(day: str) -> str:
    """특정 날짜의 캘린더 가능 시간을 확인합니다."""
    # 임시 응답 - 실제 앱에서는 실제 캘린더를 확인합니다
    return f"{day}의 가능한 시간: 오전 9:00, 오후 2:00, 오후 4:00"


@tool
class Done(BaseModel):
    """이메일이 발송되었습니다."""

    done: bool

## 이메일 어시스턴트 구축하기

라우터와 에이전트를 결합하여 이메일 어시스턴트를 구축합니다.

![agent_workflow_img](./assets/email_workflow.png)

### 라우터(Router)

라우팅 단계는 분류 결정을 처리합니다.

분류 라우터는 분류 결정에만 집중하고, 에이전트는 응답에만 집중합니다.

#### 상태(State)

에이전트를 구축할 때는 시간에 따라 추적하고자 하는 정보를 고려하는 것이 중요합니다. 이번 프로젝트에서는 LangGraph의 사전 구축된 [`MessagesState`](https://langchain-ai.github.io/langgraph/concepts/low_level/#messagesstate)를 사용합니다. `MessagesState`를 확장하여 `email_input`와 `classification_decision` 키를 추가하는 사용자 정의 `State` 객체를 정의합니다.

In [3]:
from typing import Literal

from langgraph.graph import MessagesState


class State(MessagesState):
    email_input: dict
    classification_decision: Literal["ignore", "respond", "notify"]

#### 분류 노드

분류 라우팅 로직을 포함하는 함수를 정의합니다. 
이를 위해 [`structured_outputs`](https://python.langchain.com/docs/concepts/structured_outputs/)을 사용합니다. 

`Pydantic`은 타입 힌트와 유효성 검사를 제공하기 때문에 구조화된 출력 스키마를 정의하는 데 유용합니다. 

Pydantic 모델의 설명(description)은 JSON 스키마의 일부로 LLM에 전달되어 출력 변환에 대한 정보를 제공합니다.

In [ ]:
from IPython.display import Markdown, display
from studio.src.email_assistant.prompts import (
    default_background,
    default_triage_instructions,
    triage_system_prompt,
    triage_user_prompt,
)
from studio.src.email_assistant.utils import format_email_markdown, parse_email


print("")
print("=" * 20 + " triage_system_prompt " + "=" * 20)
display(Markdown(triage_system_prompt))

print("")
print("=" * 20 + " triage_user_prompt " + "=" * 20)
display(Markdown(triage_user_prompt))

print("")
print("=" * 20 + " default_background " + "=" * 20)
display(Markdown(default_background))

print("")
print("=" * 20 + " default_triage_instructions " + "=" * 20)
display(Markdown(default_triage_instructions))


==================== triage_system_prompt ====================




< Role >
Your role is to triage incoming emails based upon instructs and background information below.
</ Role >

< Background >
{background}. 
</ Background >

< Instructions >
Categorize each email into one of three categories:
1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that worth notification but doesn't require a response
3. RESPOND - Emails that need a direct response
Classify the below email into one of these categories.
</ Instructions >

< Rules >
{triage_instructions}
</ Rules >



==================== triage_user_prompt ====================



Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}


==================== default_background ====================


 
I'm Lance, a software engineer at LangChain.



==================== default_triage_instructions ====================



Emails that are not worth responding to:
- Marketing newsletters and promotional emails
- Spam or suspicious emails
- CC'd on FYI threads with no direct questions

There are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:
- Team member out sick or on vacation
- Build system notifications or deployments
- Project status updates without action items
- Important company announcements
- FYI emails that contain relevant information for current projects
- HR Department deadline reminders
- Subscription status / renewal reminders
- GitHub notifications

Emails that are worth responding to:
- Direct questions from team members requiring expertise
- Meeting requests requiring confirmation
- Critical bug reports related to team's projects
- Requests from management requiring acknowledgment
- Client inquiries about project status or features
- Technical questions about documentation, code, or APIs (especially questions about missing endpoints or features)
- Personal reminders related to family (wife / daughter)
- Personal reminder related to self-care (doctor appointments, etc)


In [ ]:
from typing import Annotated, Literal

from pydantic import BaseModel, Field

from langchain.chat_models import init_chat_model


class RouterSchema(BaseModel):
    """읽지 않은 이메일을 분석하고 내용에 따라 처리 경로를 지정하십시오."""

    reasoning: Annotated[str, Field(description="분류의 단계별 추론 과정.")]

    classification: Annotated[
        Literal["ignore", "respond", "notify"],
        Field(
            description="이메일 분류: 관련 없는 이메일은 'ignore', "
            "답변이 필요 없는 중요한 정보는 'respond', "
            "답변이 필요한 이메일은 'notify'"
        ),
    ]


llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)
llm_router = llm.with_structured_output(RouterSchema)

In [20]:
response = llm_router.invoke("""안녕하세요,
저희 새로운 AI 기반 분석 플랫폼 출시를 알려드립니다! 저희 가치 있는 고객님을 위해 처음 3개월간 50% 할인을 제공합니다.
이 제한된 기간 동안만 이메일 구독자에게만 제공되는 특별 혜택입니다. 아래 버튼을 클릭하여 할인을 받으세요:
[지금 50% 할인 받기]
이 오퍼는 48시간 내에 만료됩니다.
감사합니다,
마케팅팀""")

print("classification:", response.classification)
print("reasoning:", response.reasoning)

classification: respond
reasoning: 이 이메일은 새로운 AI 기반 분석 플랫폼 출시와 관련된 마케팅 프로모션을 알리는 내용입니다. 할인 혜택과 제한된 기간을 강조하여 수신자가 구매를 유도하려는 상업적 목적이 명확합니다. 따라서 이 이메일은 스팸 또는 광고성 이메일로 분류되어야 하며, 적절한 대응이 필요합니다.


In [ ]:
{
    "from": "marketing@company.com",
    "to": "user@example.com",
    "subject": "New Product Launch - Special Early Bird Discount 50% Off",
    "body": "Dear Customer,\n\nWe're excited to announce the launch of our new AI-powered analytics platform! As a valued customer, we're offering you an exclusive 50% discount for the first 3 months.\n\nThis limited-time offer is only available to our email subscribers. Click below to claim your discount:\n\n[Get 50% Off Now]\n\nOffer expires in 48 hours.\n\nBest regards,\nMarketing Team",
    "received_at": "2025-10-24T09:15:00Z",
    "sender_domain": "company.com",
    "expected_category": "promotional",
}

In [ ]:
from langchain.graphs import END
from langgraph.types import Command


def triage_router(state: State) -> Command[Literal["response_agent", "__end__"]]:
    """이메일 내용을 분석하여 응답, 알림 또는 무시 여부를 결정합니다."""

    author, to, subject, email_thread = parse_email(state["email_input"])
    system_prompt = triage_system_prompt.format(
        background=default_background, triage_instructions=default_triage_instructions
    )

    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )

    result = llm_router.invoke(
        [
            ("system", system_prompt),
            ("user", user_prompt),
        ]
    )
    assert isinstance(result, RouterSchema)

    if result.classification == "respond":
        print("📧 분류: 응답 필요 - 이 이메일은 답변이 필요합니다")
        return Command(
            goto="response_agent",
            update={
                "messages": [
                    (
                        "user",
                        f"이메일에 답장하세요: \n\n{format_email_markdown(subject, author, to, email_thread)}",
                    )
                ],
                "classification_decision": result.classification,
            },
        )

    elif result.classification == "ignore":
        print("🚫 분류: 무시 - 이 이메일은 안전하게 무시해도 됩니다")
        return Command(
            goto=END, update={"classification_decision": result.classification}
        )

    elif result.classification == "notify":
        print("🔔 분류: 알림 - 이 이메일에는 중요한 정보가 포함되어 있습니다")
        return Command(
            goto=END, update={"classification_decision": result.classification}
        )

    else:
        raise ValueError(f"Invalid classification: {result.classification}")
